In [1]:
import tensorflow as tf
import numpy as np
import corpus_reader as cr #corpusを読み込むためのモジュール
import helpers #dataをバッチ化するモジュール

tf.reset_default_graph()
sess = tf.InteractiveSession()

In [3]:
# load datasets

sequences_jp, sequences_en, vocabulary_jp, vocabulary_en = cr.load_dataset('data\JEC_basic_sentence_v1-2.xls')

In [26]:
sequences_en

array([[1, 419, 3283, 4950, 2639, 2780, 3097, 1106, 763, 15, 2],
       [1, 419, 918, 4593, 763, 4972, 1106, 3218, 15, 2],
       [1, 442, 918, 4047, 2915, 2780, 2776, 4586, 15, 2], ...,
       [1, 701, 2441, 2714, 3, 3677, 2364, 2758, 4577, 1083, 4493, 15, 2],
       [1, 701, 2303, 4922, 1106, 838, 4628, 4577, 2900, 19, 2363, 179, 2],
       [1, 408, 3688, 2580, 1336, 3459, 2758, 4577, 3509, 3275, 2580, 4737, 15, 2]], dtype=object)

In [5]:
len(vocabulary_en)

5000

In [6]:
# Hyperparameter

src_vocab_size = len(vocabulary_jp)
tgt_vocab_size = len(vocabulary_en)
embedding_size = 200
encoder_num_units = 256
decoder_num_units = encoder_num_units * 2
learning_rate = 0.001

In [8]:
# Special Token

EOS = vocabulary_en.index(cr.EOS)
PAD = vocabulary_en.index(cr.PAD)

In [9]:
# input placeholder

# [max_encoder_time, batch_size]
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')

# [max_decoder_time. batch_size]
decoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_inputs')

# [max_decoder_time, batch_size]
decoder_outputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_outputs')

# [batch_size]
source_sequence_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='source_sequence_length')

# [batch_size]
decoder_lengths = tf.placeholder(shape=(None,), dtype=tf.int32, name='decoder_lengths')

In [10]:
# embedding layer

embedding_encoder = tf.get_variable(
    "embedding_encoder", [src_vocab_size, embedding_size])

# Look up embedding:
#   encoder_inputs: [max_encoder_time, batch_size]
#   encoder_emb_inp: [max_encoder_time, batch_size, embedding_size]
encoder_emb_inp = tf.nn.embedding_lookup(
    embedding_encoder, encoder_inputs)

embedding_decoder = tf.get_variable(
    "embedding_decoder", [tgt_vocab_size, embedding_size])

# Look up embedding:
#   decoder_inputs: [max_decoder_time, batch_size]
#   decoder_emb_inp: [max_decoder_time, batch_size, embedding_size]
decoder_emb_inp = tf.nn.embedding_lookup(
    embedding_decoder, decoder_inputs)

In [11]:
# encoder

# Build Encoder cell using LSTMCell
encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(encoder_num_units)

# Run Encoder with Dynamic RNN
#   encoder_outputs: [max_encoder_time, batch_size, encoder_num_units]
#   encoder_state: [batch_size, encoder_num_units]
encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_emb_inp,
    sequence_length=source_sequence_length, dtype=tf.float32, time_major=True)

In [13]:
# decoder

# Build Decoder cell using LSTMCell
decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(decoder_num_units)

# Helper
helper = tf.contrib.seq2seq.TrainingHelper(
    decoder_emb_inp, decoder_lengths, time_major=True)

# Projection Layer
projection_layer = tf.contrib.keras.layers.Dense(
    tgt_vocab_size, use_bias=False)

# Attention Layer (ただの Wx + bのレヤー)
thought_vector_h = tf.layers.dense(encoder_state.h, decoder_num_units, use_bias=True)
thought_vector_c = tf.layers.dense(encoder_state.c, decoder_num_units, use_bias=True)

thought_vector = tf.nn.rnn_cell.LSTMStateTuple(h=thought_vector_h, c=thought_vector_c)

# Decoder
decoder = tf.contrib.seq2seq.BasicDecoder(
    decoder_cell, helper, thought_vector,
    output_layer=projection_layer)

# Dynamic decoding
outputs, states, _ = tf.contrib.seq2seq.dynamic_decode(decoder, ...)
logits = outputs.rnn_output

In [14]:
# Loss Layer
crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=decoder_outputs,
    logits=logits)
loss = (tf.reduce_mean(crossent))

In [15]:
# Training Step
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [16]:
# Prediction Layer
prediction = tf.argmax(logits, 2)

In [17]:
def sequences_to_batch(sequences_src, sequences_tgt, batch_size):
    i = 0
    max_len = len(sequences_src)
    while True:
        if i + batch_size < max_len:
            yield {
                'src': sequences_src[i:i + batch_size],
                'tgt': sequences_tgt[i:i + batch_size]
            }
            i += batch_size
        else:
            yield {
                'src': (sequences_src + sequences_src)[i:i + batch_size],
                'tgt': (sequences_tgt + sequences_tgt)[i:i + batch_size]
            }
            i = (i + batch_size) % max_len

In [18]:
batch_size = 100

batches = sequences_to_batch(sequences_jp, sequences_en, batch_size)

print('head of the batch:')
for seq in next(batches)['src'][:10]:
    print(seq)
    print(cr.sequence_to_sentence(seq, vocabulary_jp, 'jp'))

head of the batch:
[1, 4, 387, 529, 439, 131, 416, 3630, 465, 2681, 2]
<s>Xではないかとつくづく疑問に思う</s>
[1, 4, 154, 41, 453, 59, 623, 2676, 602, 2]
<s>Xがいいなといつも思います</s>
[1, 323, 154, 37, 666, 473, 623, 2676, 602, 2]
<s>それがあるようにいつも思います</s>
[1, 323, 154, 2197, 439, 145, 3288, 68, 2]
<s>それが多すぎないかと正直思う</s>
[1, 2434, 529, 621, 465, 2260, 674, 915, 519, 1365, 357, 403, 2681, 2]
<s>山田はみんなに好かれるタイプの人だと思う</s>
[1, 15, 403, 4334, 154, 2683, 330, 2]
<s>〜と誰かが思った</s>
[1, 4, 543, 235, 357, 403, 2676, 602, 2]
<s>Xはしんどいことだと思います</s>
[1, 4, 529, 3076, 519, 2055, 403, 2676, 602, 2]
<s>Xは時間の問題と思います</s>
[1, 4, 529, 1386, 519, 4336, 403, 2676, 602, 2]
<s>Xは今後の課題と思います</s>
[1, 323, 529, 3211, 3066, 1763, 519, 1500, 357, 403, 2676, 602, 2]
<s>それは桃山時代前後の作品だと思います</s>


In [19]:
# 各トレーニングステップの次のデータ

def next_feed():
    batch = next(batches)
    encoder_inputs_, source_sequence_length_ = helpers.batch(batch['src'])
    decoder_inputs_, decoder_lengths_ = helpers.batch(
        [[EOS] + sequence[:-1] for sequence in batch['tgt']]
    )
    decoder_outputs_, _ = helpers.batch(batch['tgt'])
    return {
        encoder_inputs: encoder_inputs_,
        source_sequence_length: source_sequence_length_,
        decoder_inputs: decoder_inputs_,
        decoder_lengths: decoder_lengths_,
        decoder_outputs: decoder_outputs_,
    }

In [20]:
# 各トレーニングステップのlossを記録する

loss_track = []

In [21]:
# セッションを初期化

sess.run(tf.global_variables_initializer())

In [23]:
# Trainingを実施

max_batches = 3001
batches_in_epoch = 10 

try:
    for batch in range(max_batches):
        fd = next_feed()
        _, l = sess.run([train_op, loss], fd)
        loss_track.append(l)

        if batch == 0 or batch % batches_in_epoch == 0: # 各epochで状態を表示する
            print('batch {}'.format(batch))
            print('  minibatch loss: {}'.format(sess.run(loss, fd)))
            predict_ = sess.run(prediction, fd)
            for i, (inp, pred) in enumerate(zip(fd[encoder_inputs].T, predict_.T)):
                print('  sample {}:'.format(i + 1))
                print('    input     > {}'.format(cr.sequence_to_sentence(inp, vocabulary_jp, 'jp')))
                print('    predicted > {}'.format(cr.sequence_to_sentence(pred, vocabulary_en, 'en')))
                if i >= 2:
                    break
            print()

except KeyboardInterrupt:
    print('training interrupted')

batch 0
  minibatch loss: 8.501152992248535
  sample 1:
    input     > <s>血の味がします</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> <s> <s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>彼が住所変更の手続をします</s><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> <s> <s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 3:
    input     > <s>報道機関が調査をする</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> <s> <s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

batch 10
  minibatch loss: 4.30316686630249
  sample 1:
    input     > <s>Xが原因と思われます</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>どんな商売をしている人なんだろうと疑問に思われる</s><pad><pad><pad><pad>


batch 110
  minibatch loss: 2.770948886871338
  sample 1:
    input     > <s>彼がそういう人で良かった</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He <unk> <unk> <unk> . . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>この種類が一番糖度と酸味のバランスが良い</s><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He <unk> <unk> <unk> <unk> <unk> . . . . . . . </s> . <pad> <pad>
  sample 3:
    input     > <s>とても女性には評判が良い</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He <unk> <unk> <unk> . . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

batch 120
  minibatch loss: 3.1188724040985107
  sample 1:
    input     > <s>皆様の情報をお待ちします</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He is the the . . . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>楽しみに商品の到着を待ってます</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted

batch 220
  minibatch loss: 2.1953012943267822
  sample 1:
    input     > <s>施工時に注意が必要だ</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He is <unk> <unk> . . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>それがサービス計画の作成に必要だ</s><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He is <unk> <unk> the . . . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 3:
    input     > <s>その仕組みがどうしても必要だ</s><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He is <unk> the . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

batch 230
  minibatch loss: 2.7879934310913086
  sample 1:
    input     > <s>彼がゆっくりした気分で朝のひとときを過ごします</s><pad><pad><pad><pad><pad>
    predicted > <s> He will the the the the </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>百四十六名が秋のひと時を過ごしました</s><pad><pad><pad><pad>
    predicted > 

batch 340
  minibatch loss: 2.706437110900879
  sample 1:
    input     > <s>一人一人がまず年間の売上目標を立てる</s><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He will the the the the the time . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>途端に心臓がバクバクと音を立てた</s><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He 'm is the . . the . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 3:
    input     > <s>彼が身じろぎもせず、静かに寝息を立てた</s><pad><pad><pad><pad><pad><pad>
    predicted > <s> He will the to the . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

batch 350
  minibatch loss: 2.240718364715576
  sample 1:
    input     > <s>山田の手がぎゅっと私の手を握った</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He will the to the . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>彼が再びこの車の<unk>を<unk></s

batch 450
  minibatch loss: 1.9821178913116455
  sample 1:
    input     > <s>彼が心拍変動解析を用いる</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He will a in . . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>それがスカンジナビア半島の南に位置する</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He will a to the lot time of the first . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 3:
    input     > <s>それは公園の北側に位置する</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He will a of the first . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

batch 460
  minibatch loss: 2.3395144939422607
  sample 1:
    input     > <s>当社は<unk>には<unk>に現地法人を設立しました</s><pad><pad><pad><pad>
    predicted > <s> He the , the will to be . . the . </s> <pad> <pad> <pad> <pad> <pad> <p

...

batch 2460
  minibatch loss: 0.8378937244415283
  sample 1:
    input     > <s>人は常にどこかの空間の中に身を置いている</s><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> I with find a lot for themselves . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>本会に以下の委員会を置く</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> The will send the the following committees : </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 3:
    input     > <s>当委員会はXを念頭に置いております</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> X committee bears X . mind . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

batch 2470
  minibatch loss: 0.8178050518035889
  sample 1:
    input     > <s>それが更に世界に交流の輪を広げます</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> It will be expand the wave of the worldwide . </s> <pad> <pad> <

batch 2570
  minibatch loss: 0.7428710460662842
  sample 1:
    input     > <s>カタログには多数の商品をご用意しております</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> The people are available to the catalog . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>あらかじめ、いくつかのプログラムを用意する</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> The the programs beforehand . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 3:
    input     > <s>一人旅やビジネスの客様にはシングルタイプの部屋を用意します</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> The will been rooms for the traveling by themselves and <unk> November travelers . </s> <pad> <pad> <pad> <pad> <pad> <pad>

batch 2580
  minibatch loss: 0.892356812953949
  sample 1:
    input     > <s>彼がついに現役生活に別れを告げました</s><pad><pad><pad><pad

batch 2670
  minibatch loss: 0.7304550409317017
  sample 1:
    input     > <s>車が脇見運転で追突事故を起こす</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> I <unk> has a house off the road for the moment , and rear-ended someone . </s> <pad> <pad> <pad>
  sample 2:
    input     > <s>すぐになんらかの行動を起こしましょう</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> The 's get the measures immediately . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 3:
    input     > <s>歯ぐきが炎症を起こしました</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> It gums became inflamed . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

batch 2680
  minibatch loss: 0.7557111382484436
  sample 1:
    input     > <s>試合は一進一退のまま０−０で前半を終えた</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> The following of neck and neck and other total time ended in a 0-0 tie . </s> <pad> <pad> 

batch 2780
  minibatch loss: 0.7271191477775574
  sample 1:
    input     > <s>各地に導入の動きが広がった</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> The to introduce the on the areas . . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     > <s>香りがふわっと口の中に広がります</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> The weather will . the mouth . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 3:
    input     > <s>海が私の目の前に広がりました</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> I man continues her ～ eyes . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

batch 2790
  minibatch loss: 0.7348648309707642
  sample 1:
    input     > <s>Xが全く感じられません</s><pad><pad><pad><pad><pad><pad><

batch 2890
  minibatch loss: 0.6657037734985352
  sample 1:
    input     > <s>彼が毎日の配布状況を、即日または翌日の午前中までに、報告します</s>
    predicted > <s> He will the the information circulation status either that day . by the first . the first . </s> <pad> <pad>
  sample 2:
    input     > <s>彼がすみやかに経緯及び結果を委員会に報告する</s><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He will the balsam and the for . the end . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 3:
    input     > <s>彼が速やかにその結果を市長に報告する</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> He will reports the result to the mayor . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

batch 2900
  minibatch loss: 0.859882116317749
  sample 1:
    input     > <s>男がゆっくりと後ろを振り返った</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
    predicted > <s> The man slowly stood around the </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
  sample 2:
    input     >